In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
os.chdir("../")


In [2]:
data=pd.read_csv("./merged/len3_prod_mon.csv",encoding="shift-jis",index_col=0)
# drop never treated group
data=data.dropna(subset=["Approved"])
data

,year,month,id,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,self_agg,self_pro_agg,self_pro_dom,year_month,Approved,KEGG,Ingredient,variable
36,2009,1,114,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,0.0,0.0,0.0,2009-01,1985.0,D00141,Indometacin,id1
37,2009,1,114,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,0.0,0.0,0.0,2009-01,1985.0,D00126,Ibuprofen,id1
38,2009,2,114,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,0.0,0.0,0.0,2009-02,1985.0,D00141,Indometacin,id1
39,2009,2,114,解熱鎮痛消炎剤,9061579,8834341,7534695,2583588,2569825,13763,0.0,0.0,0.0,2009-02,1985.0,D00126,Ibuprofen,id1
40,2009,3,114,解熱鎮痛消炎剤,11234470,11067590,8160160,2586928,2581213,5715,0.0,0.0,0.0,2009-03,1985.0,D00141,Indometacin,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41011,2023,4,629,その他の化学療法剤,"3,835,795","2,721,078","823,165",-,-,-,0.0,0.0,0.0,2023-04,2011.0,D00282,Clotrimazole,id3
41124,2023,1,799,他に分類されない治療を主目的としない医薬品,"1,346,611","1,346,611","653,044","204,097","204,097","116,625",0.0,0.0,0.0,2023-01,2001.0,D03365,Nicotine,id1
41125,2023,2,799,他に分類されない治療を主目的としない医薬品,"1,455,085","1,455,085","820,819","248,563","248,328","127,352",0.0,0.0,0.0,2023-02,2001.0,D03365,Nicotine,id1
41126,2023,3,799,他に分類されない治療を主目的としない医薬品,"1,688,308","1,688,308","913,299","580,453","193,456",-,0.0,0.0,0.0,2023-03,2001.0,D03365,Nicotine,id1


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elasped"]=data["year"]-data["Approved"]
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
aggregated_df = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()

# dummy ver.
# aggregated_df = pd.concat([
#     data.groupby(['year', 'id']).first(), 
#     dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
# ], axis=1).reset_index()
aggregated_df

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elasped_31,elasped_32,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,elasped_39,elasped_40
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,121,1,局所麻酔剤,742998,403587,378011,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2009,123,1,自律神経剤,265974,244744,212112,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2009,124,1,鎮けい剤,1446070,1446070,944575,550,550,0,...,0,0,0,0,0,0,0,0,0,0
4,2009,131,1,眼科用剤,22936387,10895697,8144595,4663997,4663997,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,2023,441,1,抗ヒスタミン剤,"171,482","171,482","35,854","228,766","228,766","19,945",...,0,0,4,0,0,0,0,0,0,0
520,2023,449,1,その他のアレルギー用薬,"26,040,370","13,661,659","3,022,389","1,270,608","1,266,450","535,583",...,0,0,0,0,0,0,0,0,0,0
521,2023,625,1,抗ウイルス剤,"43,807,282","16,091,028","2,489,766","77,132","77,132",-,...,0,0,0,0,0,0,0,0,0,0
522,2023,629,1,その他の化学療法剤,"3,492,942","2,304,671","1,271,574",-,-,-,...,0,0,0,0,0,0,0,0,0,0


In [4]:
colnames

['elasped_m13',
 'elasped_m12',
 'elasped_m11',
 'elasped_m10',
 'elasped_m9',
 'elasped_m8',
 'elasped_m7',
 'elasped_m6',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'elasped_21',
 'elasped_22',
 'elasped_23',
 'elasped_24',
 'elasped_25',
 'elasped_26',
 'elasped_27',
 'elasped_28',
 'elasped_29',
 'elasped_30',
 'elasped_31',
 'elasped_32',
 'elasped_33',
 'elasped_34',
 'elasped_35',
 'elasped_36',
 'elasped_37',
 'elasped_38',
 'elasped_39',
 'elasped_40']

connect generic usage

In [5]:
dusage=pd.read_csv("./generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,elasped_39,elasped_40,Year,Percentage
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,2009,35.8
1,2009,121,1,局所麻酔剤,742998,403587,378011,0,0,0,...,0,0,0,0,0,0,0,0,2009,35.8
2,2009,123,1,自律神経剤,265974,244744,212112,0,0,0,...,0,0,0,0,0,0,0,0,2009,35.8
3,2009,124,1,鎮けい剤,1446070,1446070,944575,550,550,0,...,0,0,0,0,0,0,0,0,2009,35.8
4,2009,131,1,眼科用剤,22936387,10895697,8144595,4663997,4663997,0,...,0,0,0,0,0,0,0,0,2009,35.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2022,441,1,抗ヒスタミン剤,249184,249184,64226,168325,168325,7602,...,0,0,0,0,0,0,0,0,2022,79.0
486,2022,449,1,その他のアレルギー用薬,22536947,17100478,2879144,735862,665057,521264,...,0,0,0,0,0,0,0,0,2022,79.0
487,2022,625,1,抗ウイルス剤,20416443,5416725,1216460,3368,3368,-,...,0,0,0,0,0,0,0,0,2022,79.0
488,2022,629,1,その他の化学療法剤,3192222,2184317,-,-,-,-,...,0,0,0,0,0,0,0,0,2022,79.0


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,month,efficacy,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,id_321,id_332,id_339,id_391,id_399,id_441,id_449,id_625,id_629,id_799
0,2009,114,1,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,121,1,局所麻酔剤,742998,403587,378011,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2009,123,1,自律神経剤,265974,244744,212112,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2009,124,1,鎮けい剤,1446070,1446070,944575,550,550,0,...,0,0,0,0,0,0,0,0,0,0
4,2009,131,1,眼科用剤,22936387,10895697,8144595,4663997,4663997,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2022,441,1,抗ヒスタミン剤,249184,249184,64226,168325,168325,7602,...,0,0,0,0,0,1,0,0,0,0
486,2022,449,1,その他のアレルギー用薬,22536947,17100478,2879144,735862,665057,521264,...,0,0,0,0,0,0,1,0,0,0
487,2022,625,1,抗ウイルス剤,20416443,5416725,1216460,3368,3368,-,...,0,0,0,0,0,0,0,1,0,0
488,2022,629,1,その他の化学療法剤,3192222,2184317,-,-,-,-,...,0,0,0,0,0,0,0,0,1,0


In [8]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
# df_with_dummies.to_csv("./merged/len3_prod_mon_agg_drop_never.csv",encoding="shift-jis")
df_with_dummies.to_csv("./merged/len3_prod_mon_sum_agg_drop_never.csv",encoding="shift-jis")